In [ ]:
import os
import warnings

import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

In [ ]:
from darts import TimeSeries
from darts.models import (
    ARIMA,
    FFT,
    AutoARIMA,
    ExponentialSmoothing,
    LinearRegressionModel,
    NaiveDrift,
    NaiveMean,
    NaiveMovingAverage,
    NaiveSeasonal,
    Prophet,
    Theta,
)
from darts.utils.statistics import (
    check_seasonality,
    stationarity_test_adf,
)

In [ ]:
import sys

# Adicionar a pasta 'src' ao caminho do sistema
sys.path.append(os.path.abspath(os.path.join("..", "src")))

# Agora você pode importar o módulo utils Ou importar funções específicas do módulo
from metrics import (
    calculate_grouped_statistics,
    compare_series_metrics,
    sliding_window_cross_validate_and_evaluate,
)
from ts_analysis import analyze_time_series
from ts_plots import plot_forecast
from utils import convert_dfs_to_ts, preprocess_list_ts

# Params

In [ ]:
H = 1  # Horizon
K = 5 * H  # Rolling window

# Univariate Models

In [ ]:
target_columns = ["RSRP", "RSRQ", "SNR", "CQI", "RSSI"]

In [ ]:
def simple_plot(df):
    fig, ax = plt.subplots(5, 1)
    df.plot(kind="line", sharex=True, y="RSRP", ax=ax[0])
    df.plot(kind="line", sharex=True, y="RSRQ", ax=ax[1])
    df.plot(kind="line", sharex=True, y="SNR", ax=ax[2])
    df.plot(kind="line", sharex=True, y="CQI", ax=ax[3])
    df.plot(kind="line", sharex=True, y="RSSI", ax=ax[4])

## Static vs Mobile

In [ ]:
checkpoints_path = os.path.join(os.pardir, "checkpoints")

In [ ]:
df_static = pd.read_parquet(os.path.join(checkpoints_path, "5G_df_static.parquet.gzip"))

In [ ]:
len(df_static.Uid.unique().tolist())

In [ ]:
df_driving = pd.read_parquet(
    os.path.join(checkpoints_path, "5G_df_driving.parquet.gzip")
)

In [ ]:
len(df_driving.Uid.unique().tolist())

## Streaming vs. Downloading

In [ ]:
list_static_strm = df_static.query("User_Activity == 'Streaming Video'")
list_driving_strm = df_driving.query("User_Activity == 'Streaming Video'")
list_static_down = df_static.query("User_Activity == 'Downloading a File'")
list_driving_down = df_driving.query("User_Activity == 'Downloading a File'")

## Traning per unique uid

In [ ]:
def separe_for_uid(df, target_columns):
    poll = []
    uids = sorted(df.Uid.unique().tolist())
    for uid in uids:
        poll.append(df.query(f"Uid == '{uid}'")[target_columns].asfreq(freq="min"))
    return poll

In [ ]:
list_static_strm = separe_for_uid(list_static_strm, target_columns)
list_driving_strm = separe_for_uid(list_driving_strm, target_columns)
list_static_down = separe_for_uid(list_static_down, target_columns)
list_driving_down = separe_for_uid(list_driving_down, target_columns)

In [ ]:
display(
    len(list_static_strm),
    len(list_driving_strm),
    len(list_static_down),
    len(list_driving_down),
)

In [ ]:
simple_plot(list_static_strm[4])

Utils for univariate

In [ ]:
def collect_univariate_metrics(list_series, model_name, model, output_file, K=5, H=1):
    _path = os.path.join(checkpoints_path, "results", output_file)
    result_record = []
    list_transformed = preprocess_list_ts(list_series)
    for i, series in enumerate(list_transformed):
        for kpi in target_columns:
            try:
                results = sliding_window_cross_validate_and_evaluate(
                    model, series[kpi], K, H, model_name
                )
                results["target"] = kpi
                result_record.append(results)

            except Exception as e:
                print(f"Erro ao processar a serie {i} com {kpi}")
                continue

    result_record = pd.DataFrame(result_record)
    result_record.to_csv(_path)
    print(f"Saved in {_path}")
    return result_record

All list of series for training

In [ ]:
list_static_strm = convert_dfs_to_ts(list_static_strm, target_columns)
list_driving_strm = convert_dfs_to_ts(list_driving_strm, target_columns)
list_static_down = convert_dfs_to_ts(list_static_down, target_columns)
list_driving_down = convert_dfs_to_ts(list_driving_down, target_columns)

# Baseline Forecast

In [ ]:
all_stats = []

---

# Naive Forecast

In [ ]:
model_name = "Naive"
naive_model = NaiveSeasonal(K=1)

Static Streaming

In [ ]:
activity = "static_strm"
output_file = f"uni_{model_name}_{activity}.csv"


result = collect_univariate_metrics(
    list_static_strm, model_name, naive_model, output_file, K, H
)

metrics = compare_series_metrics(result)
stats = calculate_grouped_statistics(metrics)
stats["Activity"] = activity
all_stats.append(stats)
display(stats)

Driving Streaming

In [ ]:
activity = "driving_strm"
output_file = f"uni_{model_name}_{activity}.csv"


result = collect_univariate_metrics(
    list_driving_strm, model_name, naive_model, output_file, K, H
)

metrics = compare_series_metrics(result)
stats = calculate_grouped_statistics(metrics)
stats["Activity"] = activity
all_stats.append(stats)
display(stats)

Static Downloading

In [ ]:
activity = "static_down"
output_file = f"uni_{model_name}_{activity}.csv"


result = collect_univariate_metrics(
    list_static_down, model_name, naive_model, output_file, K, H
)

metrics = compare_series_metrics(result)
stats = calculate_grouped_statistics(metrics)
stats["Activity"] = activity
all_stats.append(stats)
display(stats)

Driving Downloading

In [ ]:
activity = "driving_down"
output_file = f"uni_{model_name}_{activity}.csv"


result = collect_univariate_metrics(
    list_driving_down, model_name, naive_model, output_file, K, H
)


metrics = compare_series_metrics(result)
stats = calculate_grouped_statistics(metrics)
stats["Activity"] = activity
all_stats.append(stats)
display(stats)

In [ ]:
pd.concat(all_stats).reset_index().set_index(["Model", "target", "Activity"])

---

# Naive Drift

In [ ]:
model_name = "NaiveDrift"
naive_drift_model = NaiveDrift()

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(
    list_static_strm, model_name, naive_drift_model, output_file, K, H
)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(
    list_driving_strm, model_name, naive_drift_model, output_file, K, H
)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(
    list_static_down, model_name, naive_drift_model, output_file, K, H
)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, naive_drift_model, output_file, K, H
)

# Naive Moving Average

In [ ]:
model_name = "NaiveMovingAverage"
naive_ma_model = NaiveMovingAverage(input_chunk_length=K)

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(
    list_static_strm, model_name, naive_ma_model, output_file, K, H
)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(
    list_driving_strm, model_name, naive_ma_model, output_file, K, H
)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(
    list_static_down, model_name, naive_ma_model, output_file, K, H
)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, naive_ma_model, output_file, K, H
)

# Naive Mean

In [ ]:
model_name = "NaiveMean"
naive_mean_model = NaiveMean()

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(
    list_static_strm, model_name, naive_mean_model, output_file, K, H
)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(
    list_driving_strm, model_name, naive_mean_model, output_file, K, H
)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(
    list_static_down, model_name, naive_mean_model, output_file, K, H
)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, naive_mean_model, output_file, K, H
)

# Statistical / Classic Models
---

# Exponential Smoothing

In [ ]:
model_name = "ExponentialSmoothing"
ets_model = ExponentialSmoothing()

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(list_static_strm, model_name, ets_model, output_file, K, H)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(list_driving_strm, model_name, ets_model, output_file, K, H)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(list_static_down, model_name, ets_model, output_file, K, H)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(list_driving_down, model_name, ets_model, output_file, K, H)

# Linear Regression

In [ ]:
model_name = "LinearRegression"
linear_regression_model = LinearRegressionModel(lags=K - 1)

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(
    list_static_strm, model_name, linear_regression_model, output_file, K, H
)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(
    list_driving_strm, model_name, linear_regression_model, output_file, K, H
)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(
    list_static_down, model_name, linear_regression_model, output_file, K, H
)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, linear_regression_model, output_file, K, H
)

# ARIMA

In [ ]:
model_name = "ARIMA"
arima_model = ARIMA(p=K - 1, d=1, q=2)

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(
    list_static_strm, model_name, arima_model, output_file, K=30, H=10
)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(
    list_driving_strm, model_name, arima_model, output_file, K=30, H=10
)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(
    list_static_down, model_name, arima_model, output_file, K=30, H=10
)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, arima_model, output_file, K=30, H=10
)

# Theta

In [ ]:
model_name = "Theta"
theta_model = Theta(theta=1.5)

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(
    list_static_strm, model_name, theta_model, output_file, K, H
).style.highlight_min()

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(
    list_driving_strm, model_name, theta_model, output_file, K, H
).style.highlight_min()

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(
    list_static_down, model_name, theta_model, output_file, K, H
).style.highlight_min()

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, theta_model, output_file, K=30, H=10
).style.highlight_min()

# FFT

In [ ]:
model_name = "FFT"
fft_model = FFT()

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(list_static_strm, model_name, fft_model, output_file, K, H)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(list_driving_strm, model_name, fft_model, output_file, K, H)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(list_static_down, model_name, fft_model, output_file, K, H)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, fft_model, output_file, K=30, H=10
)

# Machine Learning

---

# Prophet

In [ ]:
model_name = "Prophet"
prophet_model = Prophet()

In [ ]:
# Static Streaming
output_file = f"{model_name}_uni_static_strm.csv"
collect_univariate_metrics(
    list_static_strm, model_name, prophet_model, output_file, K, H
)

In [ ]:
# Driving Streaming
output_file = f"{model_name}_uni_driving_strm.csv"
collect_univariate_metrics(
    list_driving_strm, model_name, prophet_model, output_file, K, H
)

In [ ]:
# Static Downloading
output_file = f"{model_name}_uni_static_down.csv"
collect_univariate_metrics(
    list_static_down, model_name, prophet_model, output_file, K, H
)

In [ ]:
# Driving Downloading
output_file = f"{model_name}_uni_driving_down.csv"
collect_univariate_metrics(
    list_driving_down, model_name, prophet_model, output_file, K=30, H=10
)